In [41]:
##原始数据
import numpy as np
node_edges=np.array([[1,2,3,4,5,6,7,8,10,11,12,13,17,19,21,31],
           [0,2,3,7,13,17,19,21,30],
           [0,1,3,7,8,9,13,27,28,32],
           [0,1,2,7,12,13],
           [0,6,10],
           [0,6,10,16],
           [0,4,5,16],
           [0,1,2,3],
           [0,2,30,32,33],
           [2,33],
           [0,4,5],
           [0],
           [0,3],
           [0,1,2,3,33],
           [32,33],
           [32,33],
           [5,6],
           [0,1],
           [32,33],
           [0,1,33],
           [32,33],
           [0,1],
           [32,33],
           [25,27,29,32,33],
           [25,27,31],
           [23,24,31],
           [29,33],
           [2,23,24,33],
           [2,31,33],
           [23,26,32,33],
           [1,8,32,33],
           [0,24,25,28,32,33],
           [2,8,14,15,18,20,22,23,29,30,31,33],
           [8,9,13,14,15,18,19,20,22,23,26,27,28,29,30,31,32]])

A_mat=np.zeros(34*34).reshape(34,34)
np.set_printoptions(threshold=np.inf)  
for j in range(34):
    #print('第',j,'次更新')
    for i in node_edges[j]:
        A_mat[j][node_edges[j]]=1

In [42]:
#A_mat

In [45]:
class Modularit_community(object):
    """
    mat 表示矩阵
    max_itrs 表示最大迭代次数
    """
    def __init__(self,A_mat,optimal_or_not=False):
        self.A_mat=A_mat
        self.N = np.shape(self.A_mat)[0]
        self.B_mat=self._initialization_matrix()
        self.optimal_or_not=optimal_or_not
        
    
    def _initialization_matrix(self):
        self.k=[]
        self.k=sum(self.A_mat,0)
        self.m=0.5*sum((self.k))
        print('degree k=',self.k,'\n','m=',self.m,'\n')
        b=self.A_mat-np.dot(np.mat(self.k).T,np.mat(self.k))/(2*self.m)
        #print('b=',b)
        return b      
    
   
    def _fit(self):
        self.leading_engenvector=self._leading_eigenevctor()
        #print('leading_engenvector=',self.leading_engenvector,'\n')
        self.s,V_label,W_label=self._predict(self.leading_engenvector)
        if self._calculate_M(self.s)>0:
            print('向量s=',self.s,'\n','此时modularity=',self._calculate_M(self.s),'\n')
            info="""
            社群V={_V}
            社群W={_W}
            """.format(_V=V_label,_W=W_label)
            print(info)
        else:
            self.s=list(np.ones(len(self.B_mat)))
            print(self.s,self._calculate_M(self.s))
        return self.s
            
            
    
    def _optimal(self,optimal_or_not):      #elta=0.5
        m_tem=self._calculate_M(self.s)
        ###经过实验发现不需要开始的时候比较M是否大于零来判断是否进行，不大于零经过优化可能会大于零
        ###但是有问题的是初次划分后（非平凡解），膜小于零，优化时是的基础是以平凡解为基础还是划分的解
            
        if optimal_or_not:
            #element_max=max(list(map(abs,self.leading_engenvector))) #通过实验和常识判断，优化不需要限制点比较好
            for j in range(len(self.leading_engenvector)):
                #print('第',j,'次更新')
                for i in range(len(self.leading_engenvector)):
                    #不需要限制移动的点结果更好if abs(self.leading_engenvector[i])/element_max<=2:
                    #print('i=',i)
                    s_tem=self.s.copy()
                    s_tem[i]=self.s[i]*(-1)
                    if self._calculate_M(s_tem)>m_tem:
                        i_tem=i
                        m_tem=self._calculate_M(s_tem)
                        #print(m_tem)
                if self._calculate_M(self.s)!=m_tem:
                    self.s[i_tem]=self.s[i_tem]*(-1)
                    #print('s_tem',self.s)
                
            self.s,V_label,W_label=self._predict(self.s)
            print('向量s=',self.s,'\n','此时modularity=',self._calculate_M(self.s))
            info="""
            社群V={_V}
            社群W={_W}
            """.format(_V=V_label,_W=W_label)
            print(info)
        return self.s
      
    
    def _calculate_M(self,s):
        return np.array(s).dot(self.B_mat).dot(np.array(s))/(4*self.m)
    
    
    def _predict(self,x):
        s=[]
        V=[]
        W=[]
        for i in range(len(self.B_mat)):
            if x[i]>=0:
                s.append(1)
                V.append(i)
            else:
                s.append(-1)
                W.append(i)
        return s,V,W
    
    def _leading_eigenevctor(self):
        eigenvalue,eigenvector=np.linalg.eig(self.B_mat)
        x=eigenvector[:,eigenvalue==eigenvalue.max()]
        return x
    


In [46]:
import numpy as np
community=Modularit_community(A_mat)
s1=community._fit()
m=community.m
#community.B_mat
##标准答案是##          
##          V=[8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
##          W=[ 0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 21]

degree k= [16.  9. 10.  6.  3.  4.  4.  4.  5.  2.  3.  1.  2.  5.  2.  2.  2.  2.
  2.  3.  2.  2.  2.  5.  3.  3.  2.  4.  3.  4.  4.  6. 12. 17.] 
 m= 78.0 

向量s= [-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
 此时modularity= [[0.37146614]] 


            社群V=[8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
            社群W=[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 21]
            


In [16]:
s_ameng=community._optimal(optimal_or_not=True)
##标准答案是
##          V=[8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
##          W=[ 0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 16, 17, 19, 21]

向量s= [-1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 
 此时modularity= 0.3717948717948718

            社群V=[8, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]
            社群W=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 17, 19, 21]
            


In [6]:
eigenvalue,eigenvector=np.linalg.eig(community.B_mat)
print("""定点9连接的顶点{}
定点9对应主特征向量的分量{}
        """.format(node_edges[9],eigenvector[9,1]))


定点9连接的顶点[2, 33]
定点9对应主特征向量的分量(0.04785237690915365+0j)
        


In [24]:
B=community.B_mat
eigenvalue,eigenvector=np.linalg.eig(B)
#eigenvector[:,eigecommunitynvalue==eigenvalue.max()]
B[1].sum()

-5.551115123125783e-16

In [8]:
class reclassfy(Modularit_community):
    def __init__(self, B_ori_mat,class_result,label,team,m,optimal_or_not=False):
        self.B_mat=[]
        self.label=label
        self.m=m
        self.ori_s=class_result
        self.team=team
        for i in range(len(class_result)):
            if self.ori_s[i]==self.label:
                self.B_mat.append(B_ori_mat[i])
        self.B_mat=np.array(self.B_mat)
        for i in range(len(class_result)-1,-1,-1):
            if self.ori_s[i]!=self.label:
                self.B_mat=np.delete(self.B_mat,i,axis=1)
                #print(len(B_mat[0]),B_mat[0].sum())
        for i in range(len(self.B_mat)):
            self.B_mat[i][i]=self.B_mat[i][i]-self.B_mat[i].sum()
    
    

    def _predict(self,x):
        j=0
        V_label=[]
        W_label=[]
        s=[]
        for i in range(len(self.ori_s)):
            if self.ori_s[i]==self.label:
                if x[j]>=0:
                    s.append(1)
                    V_label.append(self.team[j])
                else:
                    s.append(-1)
                    W_label.append(self.team[j])
                j+=1      
        return s,V_label,W_label
    

In [17]:
V=[8, 9, 14, 15, 18, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]

comunity2=reclassfy(B_ori_mat=B,class_result=s1,label=1,team=V,m=m)
c2=comunity2._fit()


向量s= [-1, -1, -1, -1, -1, -1, 1, 1, 1, -1, 1, 1, -1, -1, 1, -1, -1] 
 此时modularity= 0.016765285996055226 


            社群V=[22, 23, 24, 26, 27, 30]
            社群W=[8, 9, 14, 15, 18, 20, 25, 28, 29, 31, 32]
            


In [18]:
W=[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 16, 17, 19, 21]
comunity3=reclassfy(B_ori_mat=B,class_result=s1,label=-1,team=W,m=m)
c3=comunity3._fit()

向量s= [-1, 1, 1, 1, -1, -1, -1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1] 
 此时modularity= 0.0066568047337278195 


            社群V=[1, 2, 3, 7, 9, 12, 13, 17, 19, 21]
            社群W=[0, 4, 5, 6, 10, 11, 16]
            


In [ ]:
    def _fit(self):
        self.leading_engenvector=self._leading_eigenevctor()
        #print('leading_engenvector=',self.leading_engenvector,'\n')
        self.s,V_label,W_label=self._predict(self.leading_engenvector)
        #print('s',self.s,self._calculate_M(self.s))
        if self._calculate_M(self.s)>=0:
            print('向量s=',self.s,'\n','此时modularity=',self._calculate_M(self.s),)
            info="""
            组群V={_V}
            组群W={_W}
            """.format(_V=V_label,_W=W_label)
            print(info)
        else:
            print("模块度Q值不建议划分")
            #self.s=list(np.ones(len(self.B_mat)))
            print(self.s,self._calculate_M(self.s))
        return self.s
    def _optimal(self,optimal_or_not):      
        m_tem=self._calculate_M(self.s)
        if optimal_or_not:
            #element_max=max(list(map(abs,self.leading_engenvector))) #通过实验和常识判断，优化不需要限制点比较好
            for j in range(len(self.leading_engenvector)):
                #print('第',j,'次更新')
                for i in range(len(self.leading_engenvector)):
                    #不需要限制移动的点结果更好if abs(self.leading_engenvector[i])/element_max<=2:
                    #print('i=',i)
                    s_tem=self.s.copy()
                    s_tem[i]=self.s[i]*(-1)
                    if self._calculate_M(s_tem)>m_tem:
                        i_tem=i
                        m_tem=self._calculate_M(s_tem)
                        #print(m_tem)
                if self._calculate_M(self.s)!=m_tem:
                    self.s[i_tem]=self.s[i_tem]*(-1)
                    #print('s_tem',self.s)
                
            self.s,V_label,W_label=self._predict(self.s)
            
            print('向量s=',self.s,'\n','此时modularity=',self._calculate_M(self.s))
            info="""
            组群V={_V}
            组群W={_W}
            """.format(_V=V_label,_W=W_label)
            print(info)
        return self.s

In [13]:
a=np.array([1,2,3])
np.dot(np.mat(a).T,np.mat(a))

matrix([[1, 2, 3],
        [2, 4, 6],
        [3, 6, 9]])

In [17]:
sum(A_mat,0)

array([16.,  9., 10.,  6.,  3.,  4.,  4.,  4.,  5.,  2.,  3.,  1.,  2.,
        5.,  2.,  2.,  2.,  2.,  2.,  3.,  2.,  2.,  2.,  5.,  3.,  3.,
        2.,  4.,  3.,  4.,  4.,  6., 12., 17.])

In [20]:
A_mat[0].sum()

16.0